In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import tensorflow as tf

In [4]:
df_train = pd.read_csv("train_reshape.csv",sep = ",")
df_test = pd.read_csv("test_reshape.csv",sep = ",")

df_train.drop(["Parch","Fare"],axis = 1,inplace = True)
df_test.drop(["Parch","Fare"],axis = 1,inplace = True)

df_train["Dead"] = 0.0
df_train =  df_train.iloc[:,[0,5,1,2,3,4]]
df_train["Dead"][df_train["Survived"] == 0.0] =1.0

df_test["Survived"] = 0.0
df_test["Dead"] = 0.0
df_test =  df_test.iloc[:,[4,5,0,1,2,3]]

df_train = df_train.astype("float64")
df_test = df_test.astype("float64")

In [5]:
print(df_train.head())
print(df_test.head())

   Survived  Dead  Pclass  Sex   Age  SibSp
0       0.0   1.0     3.0  0.0  22.0    1.0
1       1.0   0.0     1.0  1.0  38.0    1.0
2       1.0   0.0     3.0  1.0  26.0    0.0
3       1.0   0.0     1.0  1.0  35.0    1.0
4       0.0   1.0     3.0  0.0  35.0    0.0
   Survived  Dead  Pclass  Sex   Age  SibSp
0       0.0   0.0     3.0  0.0  34.5    0.0
1       0.0   0.0     3.0  1.0  47.0    1.0
2       0.0   0.0     2.0  0.0  62.0    0.0
3       0.0   0.0     3.0  0.0  27.0    0.0
4       0.0   0.0     3.0  1.0  22.0    1.0


In [6]:
df_test.dtypes

Survived    float64
Dead        float64
Pclass      float64
Sex         float64
Age         float64
SibSp       float64
dtype: object

In [7]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(df_train)

MinMaxScaler(copy=True, feature_range=(-1, 1))

In [8]:
df_train_norm = scaler.transform(df_train)
df_test_norm = scaler.transform(df_test)

In [9]:
df_test_norm

array([[-1.        , -1.        ,  1.        , -1.        , -0.14350339,
        -1.        ],
       [-1.        , -1.        ,  1.        ,  1.        ,  0.17064589,
        -0.75      ],
       [-1.        , -1.        ,  0.        , -1.        ,  0.54762503,
        -1.        ],
       ...,
       [-1.        , -1.        ,  1.        , -1.        , -0.04297562,
        -1.        ],
       [-1.        , -1.        ,  1.        , -1.        , -0.24974641,
        -1.        ],
       [-1.        , -1.        ,  1.        , -1.        , -0.24974641,
        -0.75      ]])

In [10]:
X_train = df_train_norm[:,2:]
Y_train = df_train_norm[:,(0,1)]
X_test = df_test_norm[:,1:]
Y_test = df_test_norm[:,(0,1)]

In [11]:
Y_train

array([[-1.,  1.],
       [ 1., -1.],
       [ 1., -1.],
       ...,
       [-1.,  1.],
       [ 1., -1.],
       [-1.,  1.]])

### TensorFlow

In [32]:
#トレーニングデータの列数
n_stocks = X_train.shape[1]
 
# ニューロンの数を設定
n_neurons_1 = 256
n_neurons_2 = 128
  
# プレースホルダーの作成（データ格納予定地的な）
X = tf.placeholder(dtype=tf.float32, shape=[None, n_stocks])
Y = tf.placeholder(dtype=tf.float32, shape=[None,2])
 
# 初期化
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

#重み、バイアスのオブジェクト(?)を生成
W_hidden_1 = tf.Variable(weight_initializer([n_stocks, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))

#同様に出力の重みも,
W_out = tf.Variable(weight_initializer([n_neurons_2, 1]))
bias_out = tf.Variable(bias_initializer([1]))

# 隠れ層の設定（ReLU＝活性化関数）
hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
 
# 出力層の設定
out = tf.transpose(tf.add(tf.matmul(hidden_2, W_out), bias_out))

#コスト関数
mse = tf.reduce_mean(tf.squared_difference(out, Y))

# 最適化関数
opt = tf.train.AdamOptimizer().minimize(mse)
    
# 初期化
net.run(tf.global_variables_initializer())

In [33]:
# ニューラルネットワークの設定
batch_size = 128
mse_train = []
mse_test = []
 
# 訓練開始！5000回の反復処理
epochs = 5000
for e in range(epochs):
    net.run(opt, feed_dict={X: X_train, Y: Y_train})
    loss_val = net.run(mse,feed_dict={X: X_train,Y:Y_train})
    #summary_writer.add_summary(loss_val,i)
    if e % 1000 == 0:
        print("Step: %d, Loss: %f" % (e,loss_val))

ValueError: Cannot feed value of shape (891, 2) for Tensor 'Placeholder_21:0', which has shape '(?,)'

In [13]:
pred_test = net.run(out, feed_dict={X: X_test})
pred_test = np.concatenate((pred_test.T, X_test), axis=1)

In [14]:
pred_inv = scaler.inverse_transform(pred_test)

In [15]:
pred_inv

array([[-2.04084516e-02,  3.00000000e+00,  0.00000000e+00,
         3.45000000e+01,  0.00000000e+00],
       [-3.60476971e-02,  3.00000000e+00,  1.00000000e+00,
         4.70000000e+01,  1.00000000e+00],
       [ 8.97931397e-01,  2.00000000e+00,  0.00000000e+00,
         6.20000000e+01,  0.00000000e+00],
       ...,
       [ 2.08662450e-02,  3.00000000e+00,  0.00000000e+00,
         3.85000000e+01,  0.00000000e+00],
       [ 1.41812235e-01,  3.00000000e+00,  0.00000000e+00,
         3.02725904e+01,  0.00000000e+00],
       [ 1.19436294e-01,  3.00000000e+00,  0.00000000e+00,
         3.02725904e+01,  1.00000000e+00]])

In [16]:
df_test.head()

,Survived,Pclass,Sex,Age,SibSp
0,0.0,3.0,0.0,34.5,0.0
1,0.0,3.0,1.0,47.0,1.0
2,0.0,2.0,0.0,62.0,0.0
3,0.0,3.0,0.0,27.0,0.0
4,0.0,3.0,1.0,22.0,1.0


In [28]:
Y_test = Y_test.reshape(Y_test.shape[0], 1)
test_inv = np.concatenate((Y_test, X_test), axis=1)

test_inv = scaler.inverse_transform(test_inv)

# テストデータの最後のデータ（正規化前）
print(df_test.values[9])
 
# テストデータの最後のデータ（正規化を戻した後）
print(test_inv[9])
 
# モデルが予測したデータ
print(pred_inv[:][9])

[ 0.  3.  0. 21.  2.]
[ 0.  3.  0. 21.  2.]
[-6.08420372e-03  3.00000000e+00  0.00000000e+00  2.10000000e+01
  2.00000000e+00]
